In [47]:
from anPysotropy.models import hudson

In [23]:
import numpy as np
import numba

In [9]:
def rot_m(t, p):
    '''
    Creates a 3x3 rotation matrix for the angles
    t (theta) and p (psi)
    '''
    R = np.zeros((3, 3))
    R[0, 0] = np.cos(p) * np.cos(t)
    R[0, 1] = -1.0 * np.sin(t)
    R[0, 2] = -1.0 * np.sin(p) * np.cos(t)
    R[1, 0] = np.cos(p) * np.sin(t)
    R[1, 1] = np.cos(t)
    R[1, 2] = -1.0 * np.sin(p) * np.sin(t)
    R[2, 0] = np.sin(p)
    R[2, 1] = 0.0
    R[2, 2] = np.cos(p)
    return R

In [29]:
def rotT(T, g):
    gg = np.outer(g, g)
    gggg = np.outer(gg, gg).reshape(4 * g.shape)
    axes = ((0, 2, 4, 6), (0, 1, 2, 3))
    return np.tensordot(gggg, T, axes)

In [12]:
def mkiso(lam, mu):
    C = np.zeros((3, 3, 3, 3))
    C[0, 0, 0, 0] = lam + 2.0 * mu
    C[1, 1, 1, 1] = lam + 2.0 * mu
    C[2, 2, 2, 2] = lam + 2.0 * mu
    C[0, 0, 1, 1] = lam
    C[0, 0, 2, 2] = lam
    C[1, 1, 0, 0] = lam
    C[1, 1, 2, 2] = lam
    C[2, 2, 0, 0] = lam
    C[2, 2, 1, 1] = lam
    C[1, 2, 1, 2] = mu
    C[1, 2, 2, 1] = mu
    C[2, 1, 1, 2] = mu
    C[2, 1, 2, 1] = mu
    C[0, 2, 0, 2] = mu
    C[0, 2, 2, 0] = mu
    C[2, 0, 0, 2] = mu
    C[2, 0, 2, 0] = mu
    C[0, 1, 0, 1] = mu
    C[0, 1, 1, 0] = mu
    C[1, 0, 0, 1] = mu
    C[1, 0, 1, 0] = mu
    return C

In [42]:
@numba.jit(nopython=True)
def Rot3333(Cijkl, R):

    rot_Cijkl = np.zeros((3, 3, 3, 3))
    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t = 0
                    for p in range(3):
                        for q in range(3):
                            for m in range(3):
                                for n in range(3):
                                    t += R[i, p] * R[j, q] * R[k, m] * R[l, n] * Cijkl[p, q, m, n]
                    rot_Cijkl[i, j, k, l] = t
    return rot_Cijkl

In [39]:
testC = mkiso(1.0, 2.0)
testR = rot_m(np.deg2rad(30), np.deg2rad(45))

In [45]:
%%time 
for i in range(1000):
    t1 = Rot3333(testC, testR)


CPU times: user 7.37 ms, sys: 364 μs, total: 7.73 ms
Wall time: 8.03 ms


In [44]:
%%time 
for i in range(1000):
    t2 = rotT(testC, testR)

CPU times: user 35.7 ms, sys: 2.34 ms, total: 38 ms
Wall time: 37 ms


In [5]:
g = rot_m(np.pi/4, np.pi/4)

In [7]:
t = np.outer(g,g)

In [8]:
t.shape

(9, 9)

In [10]:
from anPysotropy.utils.matrix_tools import voight_index_from_3x3_tensor

In [11]:
import itertools

In [19]:
for i, j, k, l in itertools.product(range(3), range(3), range(3), range(3)):
    m = voight_index_from_3x3_tensor(i, j)
    n = voight_index_from_3x3_tensor(k, l)
    print(f'{i}, {j}, {k}, {l} -> {m}, {n}')
    if m < 0 or n < 0:
        raise ValueError(f'Invalid voight index: {m}, {n}')

0, 0, 0, 0 -> 0, 0
0, 0, 0, 1 -> 0, 5
0, 0, 0, 2 -> 0, 4
0, 0, 1, 0 -> 0, 5
0, 0, 1, 1 -> 0, 1
0, 0, 1, 2 -> 0, 3
0, 0, 2, 0 -> 0, 4
0, 0, 2, 1 -> 0, 3
0, 0, 2, 2 -> 0, 2
0, 1, 0, 0 -> 5, 0
0, 1, 0, 1 -> 5, 5
0, 1, 0, 2 -> 5, 4
0, 1, 1, 0 -> 5, 5
0, 1, 1, 1 -> 5, 1
0, 1, 1, 2 -> 5, 3
0, 1, 2, 0 -> 5, 4
0, 1, 2, 1 -> 5, 3
0, 1, 2, 2 -> 5, 2
0, 2, 0, 0 -> 4, 0
0, 2, 0, 1 -> 4, 5
0, 2, 0, 2 -> 4, 4
0, 2, 1, 0 -> 4, 5
0, 2, 1, 1 -> 4, 1
0, 2, 1, 2 -> 4, 3
0, 2, 2, 0 -> 4, 4
0, 2, 2, 1 -> 4, 3
0, 2, 2, 2 -> 4, 2
1, 0, 0, 0 -> 5, 0
1, 0, 0, 1 -> 5, 5
1, 0, 0, 2 -> 5, 4
1, 0, 1, 0 -> 5, 5
1, 0, 1, 1 -> 5, 1
1, 0, 1, 2 -> 5, 3
1, 0, 2, 0 -> 5, 4
1, 0, 2, 1 -> 5, 3
1, 0, 2, 2 -> 5, 2
1, 1, 0, 0 -> 1, 0
1, 1, 0, 1 -> 1, 5
1, 1, 0, 2 -> 1, 4
1, 1, 1, 0 -> 1, 5
1, 1, 1, 1 -> 1, 1
1, 1, 1, 2 -> 1, 3
1, 1, 2, 0 -> 1, 4
1, 1, 2, 1 -> 1, 3
1, 1, 2, 2 -> 1, 2
1, 2, 0, 0 -> 3, 0
1, 2, 0, 1 -> 3, 5
1, 2, 0, 2 -> 3, 4
1, 2, 1, 0 -> 3, 5
1, 2, 1, 1 -> 3, 1
1, 2, 1, 2 -> 3, 3
1, 2, 2, 0 -> 3, 4
1, 2, 2, 1 -

In [ ]:
def voight_index_from_3x3_tensor(i, j):
    '''
    Converts indices of the full 3x3x3x3 elastic tensor
    to Voight indicies
    '''
    if i > 5 or j > 5:
        raise ValueError(f'Invalid index: {i}, {j}. Dont forget python indexes start at 0')
    if i == j:
        idx = i
    else:
        idx = 6 - i - j
    return idx

In [21]:
voight_index_from_3x3_tensor(2,4)

0